## Detector de flores utilizando Deep Learning

En esta demo construiremos un modelo para predecir el tipo de flor en una imagen utilizando Transfer Learning

<img src="https://www.pyimagesearch.com/wp-content/uploads/2014/01/flowers17sample.jpg" width="800" />

Descargaremos el set de datos desde el sitio de TensorFlow. Este archivo está comprimido, asique lo descomprimiremos

In [2]:
%sh 
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 3 218M 3 8200k 0 0 9357k 0 0:00:23 --:--:-- 0:00:23 9350k
 25 218M 25 56.0M 0 0 29.6M 0 0:00:07 0:00:01 0:00:06 29.6M
 47 218M 47 104M 0 0 37.1M 0 0:00:05 0:00:02 0:00:03 37.1M
 80 218M 80 176M 0 0 45.7M 0 0:00:04 0:00:03 0:00:01 45.7M
100 218M 100 218M 0 0 50.6M 0 0:00:04 0:00:04 --:--:-- 50.6M
tar: flower_photos/roses/14810868100_87eb739f26_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/1446090416_f0cad5fde4.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15319767030_e6c5602a77_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15032112248_30c5284e54_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/7211616670_2d49ecb3a5_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15674450867_0ced942941_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/17158274118_00ec99a23c.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14019883858_e5d2a0ec10_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8035908422_87220425d2_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14747962886_2bff6bb323_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4356781875_92c5cd93c0.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8524505546_b242bd4928_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/9406573080_60eab9278e_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/6039330368_c30ed224c4_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14414100710_753a36fce9.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/3292434691_392071d702_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5273722065_c85d8543c2_m.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/2215318403_06eb99176a.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/172311368_49412f881b.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/15255964454_0a64eb67fa.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4504220673_af754fcb40_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/505517255_cfbb6f6394.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/6347846935_51e3dc2481_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/2273917656_6d6c038283.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/8775267816_726ddc6d92_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5206847130_ee4bf0e4de_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/5777669976_a205f61e5b.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14001990976_bd2da42dbc.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/4713531680_1110a2fa07_n.jpg: Cannot change ownership to uid 270850, gid 5000: Invalid argument
tar: flower_photos/roses/14154164774_3b39d36778.jpg: Cannot change ownership to uid 2708

## Montamos nuestro "Data Lake" al cluster de Databricks

Montaremos nuestro Azure Datalake (blob storage) al file system de Databricks

<img src="https://www.talavant.com/wp-content/uploads/2019/08/Databricks-Best-Practices-Header.png" width='500' />

In [4]:
import os

if "/mnt/azureblob" not in map(lambda mnt : mnt.mountPoint, dbutils.fs.mounts()):
  dbutils.fs.mount(
    source = "wasbs://<CONTAINER>@<ACCOUNT>.blob.core.windows.net/",
    mount_point = "/mnt/azureblob",
    extra_configs = {"fs.azure.account.key.<ACCOUNT>.blob.core.windows.net":os.environ['BLOB_KEY']})

In [5]:
basepath = "/mnt/azureblob/flowers-classification/"
dbutils.fs.mkdirs(basepath)

# Small dataset created for a fast demo
#basepath_aitour = "/mnt/azureblob/flowers-classification-aitour/"
#dbutils.fs.mkdirs(basepath_aitour)

Out[4]: True

In [6]:
# Copy the dataset from the DBFS to the Azure Data Lake (execute once)
dbutils.fs.cp('file:/databricks/driver/flower_photos', basepath, recurse=True)


path,name,size
dbfs:/mnt/azureblob/flowers-classification/LICENSE.txt,LICENSE.txt,418049
dbfs:/mnt/azureblob/flowers-classification/daisy/,daisy/,0
dbfs:/mnt/azureblob/flowers-classification/dandelion/,dandelion/,0
dbfs:/mnt/azureblob/flowers-classification/roses/,roses/,0
dbfs:/mnt/azureblob/flowers-classification/sunflowers/,sunflowers/,0
dbfs:/mnt/azureblob/flowers-classification/tulips/,tulips/,0


#### Creamos nuestro set de datos

Para crearlo, leeremos recursivamente del directorio donde tenemos las images y luego agregaremos una columna llamada label que sera completada con el nombre del directorio en donde esta la imagen. El number del directorio donde esta la imagen representa el tipo de flor del que se trata. La funcion input_folder_name es definida como una función UDF

In [8]:
from pyspark.sql.types import *
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import udf

input_folder_name = udf(lambda path: path.split("/")[-2], StringType())

In [9]:
# full dataset (takes time :))
flowers_df = spark.read.load(basepath + "*/*.jpg", format="image").withColumn("label", input_folder_name(input_file_name()))

Para ahorrar tiempo, ejecutaremos el entrenamiento sobre una pequena parte del set de datos. Para esto estamos haciendo un sampleBy y solamente seleccionando un 20% de las imagenes que se corresponden con tulipanes y girasoles. Esta información la guardamos en formato parquet. No ejecute la siguiente linea si utiliza el dataset completo

In [11]:
# sampling and saving as parquet for speed sake
flowers_df = spark.read.load(basepath + "*/*.jpg", format="image").withColumn("label", input_folder_name(input_file_name())).sampleBy("label", fractions = { "tulips":0.2, "sunflowers":0.2 })
flowers_df.write.mode("Overwrite").parquet(basepath_aitour + "flowers.parquet")

#small dataset
flowers_df = spark.read.parquet(basepath_aitour + "flowers.parquet/*.parquet")

image,label
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/2431737309_1468526f8b.jpg, 441, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/4932735362_6e1017140f.jpg, 333, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/8717900362_2aa508e9e5.jpg, 333, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/4341530649_c17bbc5d01.jpg, 290, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/daisy/5693459303_e61d9a9533.jpg, 322, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",daisy
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/5674170543_73e3f403fb.jpg, 332, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/10164073235_f29931d91e.jpg, 335, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/6140892289_92805cc590.jpg, 332, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/244074259_47ce6d3ef9.jpg, 375, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/daisy/3704306975_75b74497d8.jpg, 322, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",daisy


In [12]:
display(flowers_df)

image,label
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/2431737309_1468526f8b.jpg, 441, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/4932735362_6e1017140f.jpg, 333, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/8717900362_2aa508e9e5.jpg, 333, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/4341530649_c17bbc5d01.jpg, 290, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/daisy/5693459303_e61d9a9533.jpg, 322, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",daisy
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/5674170543_73e3f403fb.jpg, 332, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/tulips/10164073235_f29931d91e.jpg, 335, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",tulips
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/6140892289_92805cc590.jpg, 332, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/sunflowers/244074259_47ce6d3ef9.jpg, 375, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",sunflowers
"List(dbfs:/mnt/azureblob/flowers-classification/daisy/3704306975_75b74497d8.jpg, 322, 500, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",daisy


#### Dividimos nuestro set de datos en train y test

Dividimos nuestro set de datos en train y test con proporciones 70% y 30% de las filas.

In [14]:
train, test = flowers_df.repartition(100, "label").randomSplit([0.7, 0.3])

### Rutina de entrenamiento

Crearemos una rutina utilizando un Pipepile que nos permite incluir varias operaciones sobre un mismo set de datos. En este caso estamos haciendo 3 operaciones:

- Utilizando un StringIndexer convertimos la variable label que es categorica
- DeepImageFeaturizer nos permite generar features basadas en un modelo de deep learning (InceptionV3) utilizando Transfer Learning
- LogisticRegression sera nuestro clasificador

##### Sobre transfer learning

Todas los objetos tienen determinados elementos en comun: podemos distinguir formas, colores, texturas, etc. Transfer Learning trata de utilizar este concepto para trasladar los conceptos que aprendio desde la tarea de deteccion de determinados para detectar objetos nuevos para los que nunca fue entrenado. Esta ultima parte se la suele referir como fine tunning

<img src="http://machinelearningcoban.com/assets/q2_tl/multi_layers.png" width="900" />

Las features que son generadas de una CNN lucen como siguien:

<img src="http://ruder.io/content/images/2017/03/example_filters.jpeg" />

In [17]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from sparkdl import DeepImageFeaturizer

model = Pipeline(stages = [StringIndexer(inputCol = "label", outputCol = "class"),
                          DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3"),
                          LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="class", family="multinomial")
                          ])

Databricks está integrado con MLFlow. Lo utilizaremos para registrar nuestro experimento

In [19]:
import mlflow

mlflow.start_run();
fittedModel = model.fit(train)

#### Evaluamos la performance del modelo

En este caso estamos midiendo Accuracy dado que es un problema de clasificación

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

tested_df = fittedModel.transform(test)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [22]:
predictionAndLabels = tested_df.select("prediction", "class").withColumnRenamed("class", "label")
acc = evaluator.evaluate(predictionAndLabels)
print("Training set accuracy = " + str(acc))

Training set accuracy = 0.8732394366197183

#### Finalizamos la ejecucion y logueamos algunas metricas mas

Utilizando la API de MLFlow, registraremos en nuestro experimento el modelo base que utilizamos, los parametros que usamos para la Logistic Regression y el Accuracy que obtuvimos. Una vez finalizado, terminamos el experimento para que quede logueado correctamente el tiempo que tardó el entrenamiento

In [24]:
mlflow.log_param("BaseModel", "InceptionV3")
mlflow.log_param("regParam", 0.05)
mlflow.log_param("elasticNetParam", 0.03)
mlflow.log_param("family", "multinomial")
mlflow.log_metric("Accuracy", acc)
mlflow.end_run()